In [ ]:
!pip install accelerate peft bitsandbytes transformers trl tqdm pandas torch transformers

In [ ]:
import pandas as pd
import torch
import pandas as pd
from util import format_prompt_test, generate, extract_text_after_tag
from tqdm import tqdm

# Load dataset
df = pd.read_csv('eval_data.csv')
df['input'] = df.apply(lambda row: format_prompt_test(row['essay'], row['prompt']), axis=1)

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

config = PeftConfig.from_pretrained("meiflwr/llama2-7b-feedback-ft")
base_model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf")
model = PeftModel.from_pretrained(base_model, "meiflwr/llama2-7b-feedback-ft")

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

config = PeftConfig.from_pretrained("meiflwr/llama2-7b-feedback-ft")
base_model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf")
model = PeftModel.from_pretrained(base_model, "meiflwr/llama2-7b-feedback-ft")
tokenizer = AutoTokenizer.from_pretrained("meiflwr/llama2-7b-feedback-ft")

In [ ]:
from transformers import pipeline
def generate(input, model, tokenizer):
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens = 170, device = 0)
    result = pipe(f"{input}")
    res = result[0]['generated_text']
    return res

In [ ]:
from tqdm import tqdm
tqdm.pandas()
df['predictions'] = df.progress_apply(lambda row: generate(row['input'], model, tokenizer), axis=1)
df['predictions'][0]